In [2]:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# Create an API client for Anthropic

from anthropic import Anthropic

client = Anthropic()
model = "claude-sonnet-4-0"

In [ ]:
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)
    
def add_assistant_message(messages, text):
    add_assistant_messages = {"role": "assistant", "content": text}
    messages.append(add_assistant_messages)
    
def chat(messages, system=None, temperature=0.7):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature
    }
    
    if system:
        params["system"] = system
    
    message = client.messages.create(**params)
    return message.content[0].text

In [ ]:
messages = []

system = """
You are a patient math tutor.
Do not directly answer a student's question.
Guide them to a solution step by step.
"""

add_user_message(messages, "How do I solve 5X+3=2 for X?")

answer = chat(messages) # or chat(messages, system=system) to include system prompt
answer

'To solve 5X + 3 = 2 for X, I need to isolate X by using inverse operations.\n\n**Step 1:** Subtract 3 from both sides\n5X + 3 - 3 = 2 - 3\n5X = -1\n\n**Step 2:** Divide both sides by 5\n5X ÷ 5 = -1 ÷ 5\nX = -1/5\n\n**Check:** Let me verify by substituting back into the original equation:\n5(-1/5) + 3 = -1 + 3 = 2 ✓\n\nTherefore, **X = -1/5** (or -0.2 in decimal form).'

In [9]:
messages = []

system = """
You are a Senior Software Engineer and Python expert. Only provide concise, relevant answers to the user's questions. 
Do not provide any additional information or context.
"""

add_user_message(messages, "Write a python function that checks a string for dupiicate characters.")

answer = chat(messages, system=system)
answer

'```python\ndef has_duplicate_chars(s):\n    return len(s) != len(set(s))\n```\n\nOr if you need to return the duplicate characters:\n\n```python\ndef find_duplicate_chars(s):\n    seen = set()\n    duplicates = set()\n    for char in s:\n        if char in seen:\n            duplicates.add(char)\n        else:\n            seen.add(char)\n    return duplicates\n```'

In [14]:
messages = []

add_user_message(messages, "Generate a one sentence movie idea")

answer = chat(messages, temperature=1.0)

answer

"A reclusive librarian discovers that every book returned late to her small-town library erases one day from the borrower's life, forcing her to choose between enforcing the rules that keep her immortal or saving her community."

In [16]:
messages = []

add_user_message(messages, "Write a one sentence description of a fake database")

stream = client.messages.create(
    model=model,   
    max_tokens=1000,
    messages=messages,
    stream=True
)

for event in stream:
    print(event)

RawMessageStartEvent(message=Message(id='msg_01Dtj7sBFtz5wf3Bx8e2GibW', content=[], model='claude-sonnet-4-20250514', role='assistant', stop_reason=None, stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=16, output_tokens=1, server_tool_use=None, service_tier='standard')), type='message_start')
RawContentBlockStartEvent(content_block=TextBlock(citations=None, text='', type='text'), index=0, type='content_block_start')
RawContentBlockDeltaEvent(delta=TextDelta(text='"', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text='CustomerInsights" is a comprehensive database', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text=' containing detailed purchasing histories, demographic profiles, and behavioral analytics', type='text_delta'), index=0, type='content_block_delta')
RawContentBlockDeltaEvent(delta=TextDelta(text=' f

In [18]:
messages = []

add_user_message(messages, "Write a one sentence description of a fake database")

with client.messages.stream(
    model=model,
    max_tokens=1000,
    messages=messages
) as stream:
    for text in stream.text_stream:
        # print(text, end='')
        pass
    
stream.get_final_message()

Message(id='msg_01MEMkB1WrQLCC8HwUpNz1a8', content=[TextBlock(citations=None, text='Here\'s a one-sentence description of a fake database:\n\nThe "Global Pet Mood Registry" is a comprehensive database containing over 847 million entries that track the daily emotional states, dream patterns, and philosophical musings of household pets across 23 countries, as reported by their owners through a sophisticated AI-powered pet telepathy interface.', type='text')], model='claude-sonnet-4-20250514', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=16, output_tokens=74, server_tool_use=None, service_tier='standard'))